## 📌 복원된 리뷰를 바탕을 난독화된 리뷰 문장 단위로 분할

- 전체 리뷰를 한번에 복원할 경우, 대응되는 부분의 길이가 일치하지 않는 문제가 주로 발생하게 됩니다.
- 따라서 전체 리뷰를 개별 문장 수준의 리뷰로 나누어 모델이 복원을 진행하도록 함으로써 이러한 문제를 최대한 완화하고자 하였습니다.
- 모델에게 개별 문장 수준에서 복원을 진행하도록 하면 난이도가 훨씬 쉬워지기 때문에 모델이 리뷰를 더 잘 복원할 수 있을거라 생각했습니다.
- 전체 리뷰를 개별 문장으로 나눌 때, 전체 리뷰를 한번에 복원한 데이터를 이용하게 됩니다.


**아래의 코드는 다음과 같은 순서로 동작합니다.**
- 전체 리뷰를 한번에 복원한 이후에 사전 기반으로 후처리한 데이터([submission_postprocess.csv](../submissions/submission_postprocess.csv))를 문장 단위로 나눕니다.
- 나뉜 문장을 바탕으로 기존 난독화된 리뷰([test.csv](../data/test.csv))에서 동일한 위치에 대응되는 부분을 나눕니다.
- 테스트 데이터를 문장 단위로 나눈 데이터([test_sentences.csv](../data/test_sentences.csv))를 저장합니다.

❗️NEXT STEP : 문장 단위로 나눈 난독화된 리뷰를 문장 단위로 복원 -> 기존 추론 코드를 참고해주세요.

### 🏷️ 데이터셋 불러오기

In [1]:
import pandas as pd

In [2]:
test_df = pd.read_csv('../data/test.csv', encoding='utf-8-sig') # 난독화된 리뷰
post_df = pd.read_csv('../submissions/submission_postprocess.csv', encoding='utf-8-sig') # 학습된 모델이 복원된 리뷰를 후처리한 리뷰 (난독화된 리뷰와 길이 일치)

In [3]:
post_df.head()

,ID,output
0,TEST_0000,너무너무 만족스러운 호텔이에요. 부산에 오면 꼭 추천하고 싶은 곳이에요. 최고입니다...
1,TEST_0001,"프론트가 없고, 조식도 없으며, 일반 입주민들이 샤있샤윔엡 있어 호텔처럼 관리가 잘..."
2,TEST_0002,진짜 불친절해요. 살면서 머물렀던 호텔 중에 최악이었습니다. 직원인지 사장인지 체크...
3,TEST_0003,뷰 맛집~~ 그런데 방음이 미흡하네요. 층간 소음과 발코니가 이중창이 아니라서 밤에...
4,TEST_0004,방 상태는 진짜 폐허 직전인데 전망은 좋아요. 보일러가 아주 찬찬하게 돌아서 추웠어...


### 🏷️ 복원된 리뷰 문장 단위로 분할

In [4]:
from kss import split_sentences

post_df['sentences'] = post_df["output"].apply(lambda x: split_sentences(x))

[Kss]: Oh! You have mecab in your environment. Kss will take this as a backend! :D



In [ ]:
post_df['sentences'].head()

0    [너무너무 만족스러운 호텔이에요., 부산에 오면 꼭 추천하고 싶은 곳이에요., 최고...
1    [프론트가 없고, 조식도 없으며, 일반 입주민들이 샤있샤윔엡 있어 호텔처럼 관리가 ...
2    [진짜 불친절해요., 살면서 머물렀던 호텔 중에 최악이었습니다., 직원인지 사장인지...
3    [뷰 맛집~~ 그런데 방음이 미흡하네요., 층간 소음과 발코니가 이중창이 아니라서 ...
4    [방 상태는 진짜 폐허 직전인데 전망은 좋아요., 보일러가 아주 찬찬하게 돌아서 추...
Name: sentences, dtype: object

### 🏷️ 복원된 리뷰 문장을 바탕으로 난독화된 리뷰를 문장 단위로 분할

In [6]:
test_sentences = {"ID" : [], "input": [], "start_idx": [], "end_idx": []}

for idx, row in post_df.iterrows():
    start_idx, end_idx = 0, 0
    for i, sent in enumerate(row['sentences']):
        start_idx = row['output'].find(sent, end_idx) # 이전 문장의 끝 인덱스부터 시작
        end_idx = start_idx + len(sent) # 시작 인덱스부터 문장 길이만큼 더함
        mapping_sent = test_df['input'][idx][start_idx:end_idx] # 난독화된 리뷰에서 복원된 리뷰의 문장과 일치하는 부분을 찾음

        test_sentences['ID'].append(row['ID'])
        test_sentences['input'].append(mapping_sent)
        test_sentences['start_idx'].append(start_idx)
        test_sentences['end_idx'].append(end_idx)

In [7]:
test_sentences_df = pd.DataFrame(test_sentences)
test_sentences_df.head()

,ID,input,start_idx,end_idx
0,TEST_0000,녀뮨넒뭅 만죡숭러윤 효템뤼에오.,0,17
1,TEST_0000,푸싸눼 옰면 콕 츄쩐학꼬 싶은 콧쉰웨오.,18,40
2,TEST_0000,췌꾜윕뉘댜! ㅎㅎ,41,50
3,TEST_0000,당음웨 또 옭 컷 갗았요.,51,65
4,TEST_0001,"풀룐투갸 엎코, 좀식또 업읍머, 윌뱐 잎츔민든릿 샤있샤윔엡 위썬 호뗄첨렴 관뤽갉 찰...",0,58


In [8]:
len(test_sentences_df)

8153

### 🏷️ 문장 단위로 분리된 난독화된 리뷰 저장

In [9]:
test_sentences_df.to_csv('../data/test_sentences.csv', index=False, encoding='utf-8-sig')